In [ ]:
import mitsuba as mi
mi.set_variant("cuda_ad_rgb")

import numpy as np
import matplotlib.pyplot as plt

import nbimporter
nbimporter.options['only_defs'] = False

import importlib

import testing_scene as ts
importlib.reload(ts)

import vapl_utils as utils
importlib.reload(utils)


In [ ]:
def weighted_loss(real, predicted, weight):
    eps = 0.01
    mse = (real - predicted) ** 2
    norm_factor = (weight * ((predicted ** 2).detach() + eps))
    return (mse / (norm_factor +  eps)).mean()

loss_function = utils.Loss(weighted_loss)
field = utils.vapl_grid(ts.scene.bbox().min, ts.scene.bbox().max, 1, 4, 8).cuda()
rhs_integrator = utils.RHSIntegrator(field, loss_function)

def should_render(epoch):
    if epoch < 50:
        return epoch % 5 == 0
    elif epoch < 500:
        return epoch % 20 == 0
    elif epoch < 2000:
        return epoch % 100 == 0
    else:
        return epoch % 250 == 0

for epoch in range(101):
    rhs_integrator.epoch = epoch

    rhs_image = mi.render(ts.scene, spp=1, integrator=rhs_integrator)

    if (should_render(epoch)):    
        fig, ax = plt.subplots()
        ax.imshow(np.clip(rhs_image ** (1.0 / 2.2), 0, 1))
        ax.axis("off")
        ax.set_title(f"VAPL epoch:{epoch}")
        plt.show()
